In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
stations = pd.read_csv("stations_with_routes.csv")
stations_gdf = gpd.GeoDataFrame(stations)
stations_gdf.head(5)

,Nimi,FMSID,LPNN,WMO,Lat,Lon,Korkeus,Ryhmät,Alkaen,geometry,buffer1000,buffer500,buffer100,Reitit(1000m),Reitit(500m),Reitit(100m),count
0,Virolahti Koivuniemi Ääpälä,100656,NaN,NaN,60.53,27.68,5.0,ilmanlaatu (IL),2013,POINT (537320.0664718065 6710631.271684047),"POLYGON ((538320.0664718065 6710631.271684047,...","POLYGON ((537820.0664718065 6710631.271684047,...","POLYGON ((537420.0664718065 6710631.271684047,...",14,9,2,3
1,Espoo Leppävaara Läkkisepänkuja,100691,NaN,NaN,60.22,24.81,9.0,ilmanlaatu (kaupungit),2010,POINT (378673.7750963532 6677925.3637112565),POLYGON ((379673.7750963532 6677925.3637112565...,POLYGON ((379173.7750963532 6677925.3637112565...,POLYGON ((378773.7750963532 6677925.3637112565...,107,38,4,8
2,Harjavalta Pirkkala,103143,NaN,NaN,61.33,22.14,40.0,ilmanlaatu (kaupungit),1990,POINT (240042.64854923848 6809227.108649717),POLYGON ((241042.64854923848 6809227.108649717...,POLYGON ((240542.64854923848 6809227.108649717...,POLYGON ((240142.64854923848 6809227.108649717...,40,18,5,2
3,Helsinki Mäkelänkatu,100762,NaN,NaN,60.20,24.95,25.0,ilmanlaatu (kaupungit),2015,POINT (386358.8842993917 6675449.520309521),"POLYGON ((387358.8842993917 6675449.520309521,...","POLYGON ((386858.8842993917 6675449.520309521,...","POLYGON ((386458.8842993917 6675449.520309521,...",28,0,0,12
4,Helsinki Vartiokylä Huivipolku,100803,NaN,NaN,60.22,25.10,25.0,ilmanlaatu (kaupungit),2009,POINT (394736.59373211383 6677427.550134117),POLYGON ((395736.59373211383 6677427.550134117...,POLYGON ((395236.59373211383 6677427.550134117...,POLYGON ((394836.59373211383 6677427.550134117...,140,48,0,2


In [3]:
gdfs = []
dfs= []

import sys
sys.path.append('../../code')
from helpers.load_routes import load_routes
load_routes(gdfs, dfs)
all_routes_gdf = gpd.GeoDataFrame(pd.concat(gdfs))

In [4]:
len(all_routes_gdf)

485227

In [10]:
# Laske reitit kullekin raja-arvoja ylittäneelle havaintoasemlle
from helpers.calculate_routes import calculate_routes
from shapely.geometry import Point
routes_3067 = all_routes_gdf




cols = routes_3067.columns

route_dfs = []

# ~5 min per asema
for i in range(len(stations_gdf)):
    city_routes = pd.DataFrame(columns=cols)
    city_routes["1000m"] = ""
    city_routes["500m"] = ""
    city_routes["100m"] = ""
    lon = stations_gdf["Lon"].iloc[i]
    lat = stations_gdf["Lat"].iloc[i]
    geometry = [Point(lon, lat)]
    city_gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
    city_3067 = city_gdf
    calculate_routes(routes_3067, city_3067, city_routes)
    city_routes = gpd.GeoDataFrame(city_routes, crs="EPSG:3067")
    city_routes = city_routes.to_crs("EPSG:4326")
    city_routes["Havaintoaseman nimi"] = stations_gdf["Nimi"].iloc[i]
    route_dfs.append(city_routes)


In [13]:
len(route_dfs)

26

In [17]:
stations_routes = gpd.GeoDataFrame(pd.concat(route_dfs))

In [19]:
export = stations_routes
export.to_csv("routes_near_all_stations.csv", index=False)